In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
# from fastai.data.core import DataLoaders
from fastai.callback.tracker import SaveModelCallback

# from bigearthnet19.bigearthnet19_utils import TrainValidCSV, GenerateBigEarthNetPNG
#from fastai.metrics import accuracy_multi

if torch.cuda.is_available():print("Great! Good to go!. You have {} GPUs".format(torch.cuda.device_count()))
else: print('CUDA is not up!')

Great! Good to go!. You have 4 GPUs


In [ ]:
from bigearthnet19.bigearthnet19_utils import TrainValidCSV, GenerateBigEarthNetPNG

dataset_dir = '/home/atsumilab/Clifford/rs_imagery/BigEarthNet-v1.0/'
patch_names = dataset_dir+'splits/test.csv'
labels = dataset_dir+'label_indices.json'
out_dir = '/home/atsumilab/Clifford/rs_imagery/BigEarthNet19/test/'

In [ ]:
TrainValidCSV(dataset_dir, patch_names, labels).csv_generation()

GenerateBigEarthNetPNG(dataset_dir, out_dir, 'rgb', patch_names, labels).generate_png()

In [62]:
dpath = Path('/home/atsumilab/Clifford/rs_imagery/BigEarthNet19/')

df = pd.read_csv(dpath/'test.csv', sep=';',  header=0, names=['keys', 'labels', 'is_valid'])

In [65]:
df.head()

,keys,labels,is_valid
0,S2A_MSIL2A_20170717T113321_28_88.png,Pastures,True
1,S2A_MSIL2A_20170717T113321_29_86.png,Pastures,True
2,S2A_MSIL2A_20170717T113321_30_79.png,Pastures,True
3,S2A_MSIL2A_20170717T113321_30_89.png,"Pastures|Land principally occupied by agriculture, with significant areas of natural vegetation",True
4,S2A_MSIL2A_20170717T113321_31_81.png,"Pastures|Land principally occupied by agriculture, with significant areas of natural vegetation",True


In [64]:
df['is_valid'] = True

In [66]:
df.to_csv('test.csv', sep=';', index=False)

In [ ]:
# get_x: lambda x: x['keys']
# get_y: lambda y: y['labels'].split('|')
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train, valid

def get_x(r): return dpath/'train'/r['keys']
def get_y(r): return r['labels'].split('|')

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y)

# dsets = dblock.datasets(df)
dls = dblock.dataloaders(df)

dls.train[0]

In [ ]:
dls.show_batch(nrows=1, ncols=1)